<a href="https://colab.research.google.com/github/poornima-bhatia/ES666-Assignment3/blob/main/assignment-3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
import glob

In [ ]:
class PanaromaStitcher():
    def _init_(self):
        pass
    def keypoints_descriptors(self,image):
        sift = cv2.SIFT_create()
        keypoints, descriptors = sift.detectAndCompute(image, None)
        descriptors = descriptors.astype(np.uint8)
        return keypoints, descriptors

    def source_destination_points(self,image1, image2):
        gray1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
        gray2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)
        keypoints1, descriptors1 = self.keypoints_descriptors(gray1)
        keypoints2, descriptors2 = self.keypoints_descriptors(gray2)
        bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
        matches = bf.match(descriptors1, descriptors2)
        matches = sorted(matches, key=lambda x: x.distance)
        draw_matching = cv2.drawMatches(image1, keypoints1, image2, keypoints2, matches, outImg=None)
        source_points = np.float32([keypoints1[m.queryIdx].pt for m in matches]).reshape(-1, 2)
        destination_points = np.float32([keypoints2[m.trainIdx].pt for m in matches]).reshape(-1, 2)
        return source_points, destination_points, draw_matching

    def homography(self,source_points, destination_points):
        num_points = source_points.shape[0]
        A_matrix = []
        for i in range(num_points):
            src_x, src_y = source_points[i, 0], source_points[i, 1]
            dst_x, dst_y = destination_points[i, 0], destination_points[i, 1]
            A_matrix.append([src_x, src_y, 1, 0, 0, 0, -dst_x * src_x, -dst_x * src_y, -dst_x])
            A_matrix.append([0, 0, 0, src_x, src_y, 1, -dst_y * src_x, -dst_y * src_y, -dst_y])
        A_matrix = np.asarray(A_matrix)
        U, S, Vh = np.linalg.svd(A_matrix) # perform SVD
        L = Vh[-1, :] / Vh[-1, -1]
        H = L.reshape(3, 3)
        return H

    def ransac(self,source_points, destination_points):
        n = source_points.shape[0]
        _H = None
        max_inliers = 0
        threshold = 0.5
        for i in range(10000):
            idx = np.random.choice(n, 4, replace=False)
            H = self.homography(source_points[idx], destination_points[idx])
            source_points_hat = np.hstack((source_points, np.ones((n, 1))))
            destination_points_hat = np.hstack((destination_points, np.ones((n, 1))))
            destination_points_hat_hat = np.matmul(H, source_points_hat.T).T
            destination_points_hat_hat = destination_points_hat_hat[:, :2] / destination_points_hat_hat[:, 2:]
            diff = np.linalg.norm(destination_points_hat_hat - destination_points, axis=1)
            inliers = np.sum(diff < threshold)
            if inliers > max_inliers:
                max_inliers = inliers
                _H = H
        return _H
    def edges_images(self,image, H):
        factor = 4
        h, w, _ = image.shape
        _h, _w = factor*h, factor*w
        edge_image = np.zeros((_h, _w, 3), dtype=np.uint8)
        for y in range(-h, _h-h):
            for x in range(-w, _w-w):
                pt = np.dot(H, np.array([x, y, 1]))
                pt = pt / pt[2]
                if 0 <= pt[0] < image.shape[1] and 0 <= pt[1] < image.shape[0]:
                    x0, y0 = int(pt[0]), int(pt[1])
                    x1, y1 = x0 + 1, y0 + 1
                    alpha = pt[0] - x0
                    beta = pt[1] - y0
                    if 0 <= x0 < image.shape[1] and 0 <= x1 < image.shape[1] and \
                    0 <= y0 < image.shape[0] and 0 <= y1 < image.shape[0]:
                        interpolated_color = (1 - alpha) * (1 - beta) * image[y0, x0] + \
                                            alpha * (1 - beta) * image[y0, x1] + \
                                            (1 - alpha) * beta * image[y1, x0] + \
                                            alpha * beta * image[y1, x1]
                        edge_image[y+h, x+w] = interpolated_color.astype(np.uint8)
        return edge_image, h, w

    # function to remove the black background
    def remove_black_background(self,img):
        mask = img.sum(axis=2) > 0
        y, x = np.where(mask)
        x_min, x_max = x.min(), x.max()
        y_min, y_max = y.min(), y.max()
        img_cropped = img[y_min:y_max+1, x_min:x_max+1, :]
        return img_cropped

    # function to align the images
    def combined_images(self,img1, img2, H, focus=2, blend=True):
        h1, w1 = img1.shape[:2]
        h2, w2 = img2.shape[:2]
        corners1 = np.array([[0, 0], [0, h1], [w1, h1], [w1, 0]], dtype=np.float32)
        corners2 = np.array([[0, 0], [0, h2], [w2, h2], [w2, 0]], dtype=np.float32)

        corners2_transformed = cv2.perspectiveTransform(corners2.reshape(1, -1, 2), H).reshape(-1, 2)
        corners = np.concatenate((corners1, corners2_transformed), axis=0)
        x_min, y_min = np.int32(corners.min(axis=0).ravel() - 0.5)
        x_max, y_max = np.int32(corners.max(axis=0).ravel() + 0.5)

        T = np.array([[1, 0, -x_min], [0, 1, -y_min], [0, 0, 1]])
        H_inv = np.linalg.inv(T.dot(H)) # inverse of the homography matrix

        if focus == 1:
            img_transformed, h_, w_ = self.edges_images(img2, H)
            img_result = img_transformed.copy()
            img_result[h_ : h1 + h_, w_ : w1 + w_] = img1

            if blend:
                img_re_edge = img_result[h_ : h_ + h1 , -5 + w_ : w_ + 5]
                img_result[h_:h_+h1 , -5 + w_ : w_ + 5] = cv2.GaussianBlur(img_re_edge, (3, 1), 5, 5)
                img_re_edge = img_result[h_ : h_ + h1 , w2 - 5 + w_ : w_ + 5 + w2]
                img_result[h_ : h_ + h1 , w2 - 5 + w_ : w_ + 5 + w2] =  cv2.GaussianBlur(img_re_edge, (3, 1), 5, 5)
                img_re_edge = img_result[-5 + h_ : h_ + 5 , w_ : w_ + w1]
                img_result[-5 + h_ : h_ + 5 , w_ : w_ + w1] =  cv2.GaussianBlur(img_re_edge, (1, 3), 5, 5)
                img_re_edge = img_result[h2 - 5 + h_ : h_ + 5  + h2, + w_ : w_ + w1]
                img_result[h2 - 5 + h_ : h_ + 5  + h2, w_ : w_ + w1] =  cv2.GaussianBlur(img_re_edge, (1, 3), 5, 5)

        else:
            img_transformed, h_, w_ = self.edges_images(img1, H_inv)
            img_result = img_transformed.copy()
            img_result[-y_min + h_ : h2 - y_min + h_, -x_min + w_ : w2 - x_min + w_] = img2

            if blend:
                img_re_edge = img_result[-y_min + h_ : h_ + h2 - y_min, -x_min - 5 + w_ : w_ + 5 - x_min]
                img_result[-y_min + h_:h_+h2 - y_min, -x_min -5+ w_:w_+5 - x_min] = cv2.GaussianBlur(img_re_edge, (3, 1), 5, 5)
                img_re_edge = img_result[-y_min + h_ : h_ + h2 - y_min, -x_min + w2 - 5+  w_ : w_ + 5 - x_min + w2]
                img_result[-y_min + h_ : h_ + h2 - y_min, - x_min + w2 - 5+ w_ : w_ + 5 - x_min + w2] =  cv2.GaussianBlur(img_re_edge, (3, 1), 5, 5)
                img_re_edge = img_result[-y_min - 5 + h_ : h_ + 5 - y_min, -x_min + w_ : w_ + w2 - x_min]
                img_result[-y_min - 5 + h_ : h_ + 5 - y_min, - x_min + w_ : w_ + w2 - x_min] =  cv2.GaussianBlur(img_re_edge, (1, 3), 5, 5)
                img_re_edge = img_result[-y_min + h2 - 5 + h_ : h_ + 5 - y_min + h2, - x_min + w_ : w_ + w2 - x_min]
                img_result[-y_min + h2 - 5 + h_ : h_ + 5 - y_min + h2, -x_min + w_ : w_ + w2 - x_min] =  cv2.GaussianBlur(img_re_edge, (1, 3), 5, 5)

        return img_result

In [ ]:
def show_image(img, w=0, h=0, axis=False, title=''):
    if w == 0 or h == 0: plt.figure()
    else: plt.figure(figsize=(w, h)) # set the size of the figure
    plt.title(title) # set the title of the figure
    plt.imshow(img) # show the image
    if axis == False: plt.axis('off') # turn off the axis
    plt.show()
def open_resize(img, w, h):
    image = cv2.imread(img) # read the image
    image = cv2.resize(image, (w, h)) # resize the image
    return image

In [ ]:
fun = PanaromaStitcher()

### path => I1

In [ ]:
path1 = '/content/drive/MyDrive/Colab Notebooks/content/Images/I1/STA_0031.JPG'
path2 = '/content/drive/MyDrive/Colab Notebooks/content/Images/I1/STB_0032.JPG'
path3 = '/content/drive/MyDrive/Colab Notebooks/content/Images/I1/STC_0033.JPG'
path4 = '/content/drive/MyDrive/Colab Notebooks/content/Images/I1/STD_0034.JPG'
path5 = '/content/drive/MyDrive/Colab Notebooks/content/Images/I1/STE_0035.JPG'
path6 = '/content/drive/MyDrive/Colab Notebooks/content/Images/I1/STF_0036.JPG'

In [ ]:
image1 = open_resize(path1, 600 , 400)
image2 = open_resize(path2, 600 , 400)
image3 = open_resize(path3, 600 , 400)
image4 = open_resize(path4, 600 , 400)
image5 = open_resize(path5, 600 , 400)
image6 = open_resize(path6, 600 , 400)
show_image(image1, axis=True)
show_image(image2, axis=True)
show_image(image3, axis=True)
show_image(image4, axis=True)
show_image(image5, axis=True)
show_image(image6, axis=True)

In [ ]:
homography_matrix_list = []

In [ ]:
source_points_1, destination_points_1, matching_result_1 = fun.source_destination_points(image1, image2)
_H1 = fun.ransac(source_points_1, destination_points_1)
mergered_image_1 = fun.combined_images(image1, image2, _H1, focus=2, blend=True)
mergered_image_1 = fun.remove_black_background((mergered_image_1))
homography_matrix_list.append(_H1)
print(_H1)
show_image(mergered_image_1, axis=True)

In [ ]:
source_points_2, destination_points_2, matching_result_2 = fun.source_destination_points(mergered_image_1, image3)
_H2 = fun.ransac(source_points_2, destination_points_2)
mergered_image_2 = fun.combined_images(mergered_image_1, image3, _H2, focus=2, blend=True)
mergered_image_2 = fun.remove_black_background((mergered_image_2))
homography_matrix_list.append(_H2)
print(_H2)
show_image(mergered_image_2, axis=True)

In [ ]:
source_points_3, destination_points_3, matching_result_3 = fun.source_destination_points(mergered_image_2, image4)
_H3 = fun.ransac(source_points_3, destination_points_3)
mergered_image_3 = fun.combined_images(mergered_image_2, image4, _H3, focus=2, blend=True)
mergered_image_3 = fun.remove_black_background((mergered_image_3))
homography_matrix_list.append(_H3)
print(_H3)
show_image(mergered_image_3, axis=True)

In [ ]:
source_points_4, destination_points_4, matching_result_4 = fun.source_destination_points(mergered_image_3, image5)
_H4 = fun.ransac(source_points_4, destination_points_4)
mergered_image_4 = fun.combined_images(mergered_image_3, image5, _H4, focus=2, blend=True)
mergered_image_4 = fun.remove_black_background((mergered_image_4))
homography_matrix_list.append(_H4)
print(_H4)
show_image(mergered_image_4, axis=True)

In [ ]:
source_points_5, destination_points_5, matching_result_5 = fun.source_destination_points(mergered_image_4, image6)
_H5 = fun.ransac(source_points_5, destination_points_5)
mergered_image_5 = fun.combined_images(mergered_image_4, image6, _H5, focus=2, blend=True)
mergered_image_5 = fun.remove_black_background((mergered_image_5))
homography_matrix_list.append(_H5)
print(_H5)
show_image(mergered_image_5, axis=True)

## Final I1

In [ ]:
print((homography_matrix_list))
show_image(mergered_image_5, axis=False)

### path => I2

In [ ]:
path1 = '/content/drive/MyDrive/Colab Notebooks/content/Images/I2/2_1.JPG'
path2 = '/content/drive/MyDrive/Colab Notebooks/content/Images/I2/2_2.JPG'
path3 = '/content/drive/MyDrive/Colab Notebooks/content/Images/I2/2_3.JPG'
path4 = '/content/drive/MyDrive/Colab Notebooks/content/Images/I2/2_4.JPG'
path5 = '/content/drive/MyDrive/Colab Notebooks/content/Images/I2/2_5.JPG'
image1 = open_resize(path1, 600 , 400)
image2 = open_resize(path2, 600 , 400)
image3 = open_resize(path3, 600 , 400)
image4 = open_resize(path4, 600 , 400)
image5 = open_resize(path5, 600 , 400)
show_image(image1, axis=True)
show_image(image2, axis=True)
show_image(image3, axis=True)
show_image(image4, axis=True)
show_image(image5, axis=True)
homography_matrix_list = []

In [ ]:
source_points_1, destination_points_1, matching_result_1 = fun.source_destination_points(image1, image2)
_H1 = fun.ransac(source_points_1, destination_points_1)
mergered_image_1 = fun.combined_images(image1, image2, _H1, focus=2, blend=True)
mergered_image_1 = fun.remove_black_background((mergered_image_1))
homography_matrix_list.append(_H1)
print(_H1)
show_image(mergered_image_1, axis=True)

In [ ]:
source_points_2, destination_points_2, matching_result_2 = fun.source_destination_points(image3, image4)
_H2 = fun.ransac(source_points_2, destination_points_2)
mergered_image_2 = fun.combined_images(image3, image4, _H2, focus=2, blend=True)
mergered_image_2 = fun.remove_black_background((mergered_image_2))
homography_matrix_list.append(_H2)
print(_H2)
show_image(mergered_image_2, axis=True)

In [ ]:
source_points_3, destination_points_3, matching_result_3 = fun.source_destination_points(mergered_image_2, image5)
_H3 = fun.ransac(source_points_3, destination_points_3)
mergered_image_3 = fun.combined_images(mergered_image_2, image5, _H3, focus=1 , blend=True)
mergered_image_3 = fun.remove_black_background((mergered_image_3))
homography_matrix_list.append(_H3)
print(_H3)
show_image(mergered_image_3, axis=True)

In [ ]:
source_points_4, destination_points_4, matching_result_4 = fun.source_destination_points(mergered_image_1, mergered_image_3)
_H4 = fun.ransac(source_points_4, destination_points_4)
mergered_image_4 = fun.combined_images(mergered_image_1, mergered_image_3, _H4, focus=1, blend=False)
mergered_image_4 = fun.remove_black_background((mergered_image_4))
homography_matrix_list.append(_H4)
print(_H4)
show_image(mergered_image_4, axis=True)

## Final I2

In [ ]:
print((homography_matrix_list))
show_image(mergered_image_4, axis=False)

### path => I3

In [ ]:
path1 = '/content/drive/MyDrive/Colab Notebooks/content/Images/I3/3_1.JPG'
path2 = '/content/drive/MyDrive/Colab Notebooks/content/Images/I3/3_2.JPG'
path3 = '/content/drive/MyDrive/Colab Notebooks/content/Images/I3/3_3.JPG'
path4 = '/content/drive/MyDrive/Colab Notebooks/content/Images/I3/3_4.JPG'
path5 = '/content/drive/MyDrive/Colab Notebooks/content/Images/I3/3_5.JPG'
image1 = open_resize(path1, 600 , 400)
image2 = open_resize(path2, 600 , 400)
image3 = open_resize(path3, 600 , 400)
image4 = open_resize(path4, 600 , 400)
image5 = open_resize(path5, 600 , 400)
show_image(image1, axis=True)
show_image(image2, axis=True)
show_image(image3, axis=True)
show_image(image4, axis=True)
show_image(image5, axis=True)
homography_matrix_list = []

In [ ]:
source_points_1, destination_points_1, matching_result_1 = fun.source_destination_points(image1, image2)
_H1 = fun.ransac(source_points_1, destination_points_1)
mergered_image_1 = fun.combined_images(image1, image2, _H1, focus=2, blend=True)
mergered_image_1 = fun.remove_black_background((mergered_image_1))
homography_matrix_list.append(_H1)
print(_H1)
show_image(mergered_image_1, axis=True)

In [ ]:
source_points_2, destination_points_2, matching_result_2 = fun.source_destination_points(image4, image5)
_H2 = fun.ransac(source_points_2, destination_points_2)
mergered_image_2 = fun.combined_images(image4, image5, _H2, focus=2, blend=True)
mergered_image_2 = fun.remove_black_background((mergered_image_2))
homography_matrix_list.append(_H2)
print(_H2)
show_image(mergered_image_2, axis=True)

In [ ]:
source_points_3, destination_points_3, matching_result_3 = fun.source_destination_points(image3, mergered_image_2)
_H3 = fun.ransac(source_points_3, destination_points_3)
mergered_image_3 = fun.combined_images(image3, mergered_image_2, _H3, focus=1 , blend=True)
mergered_image_3 = fun.remove_black_background((mergered_image_3))
homography_matrix_list.append(_H3)
print(_H3)
show_image(mergered_image_3, axis=True)

In [ ]:
source_points_4, destination_points_4, matching_result_4 = fun.source_destination_points(mergered_image_1, mergered_image_3)
_H4 = fun.ransac(source_points_4, destination_points_4)
mergered_image_4 = fun.combined_images(mergered_image_1, mergered_image_3, _H4, focus=1, blend=False)
mergered_image_4 = fun.remove_black_background((mergered_image_4))
homography_matrix_list.append(_H4)
print(_H4)
show_image(mergered_image_4, axis=True)

## Final I3

In [ ]:
print((homography_matrix_list))
show_image(mergered_image_4, axis=False)

### path => I4

In [ ]:
path1 = '/content/drive/MyDrive/Colab Notebooks/content/Images/I4/DSC02930.JPG'
path2 = '/content/drive/MyDrive/Colab Notebooks/content/Images/I4/DSC02931.JPG'
path3 = '/content/drive/MyDrive/Colab Notebooks/content/Images/I4/DSC02932.JPG'
path4 = '/content/drive/MyDrive/Colab Notebooks/content/Images/I4/DSC02933.JPG'
path5 = '/content/drive/MyDrive/Colab Notebooks/content/Images/I4/DSC02934.JPG'
image1 = open_resize(path1, 600 , 400)
image2 = open_resize(path2, 600 , 400)
image3 = open_resize(path3, 600 , 400)
image4 = open_resize(path4, 600 , 400)
image5 = open_resize(path5, 600 , 400)
show_image(image1, axis=True)
show_image(image2, axis=True)
show_image(image3, axis=True)
show_image(image4, axis=True)
show_image(image5, axis=True)
homography_matrix_list = []

In [ ]:
source_points_1, destination_points_1, matching_result_1 = fun.source_destination_points(image1, image2)
_H1 = fun.ransac(source_points_1, destination_points_1)
mergered_image_1 = fun.combined_images(image1, image2, _H1, focus=2, blend=True)
mergered_image_1 = fun.remove_black_background((mergered_image_1))
homography_matrix_list.append(_H1)
print(_H1)
show_image(mergered_image_1, axis=True)

In [ ]:
source_points_2, destination_points_2, matching_result_2 = fun.source_destination_points(mergered_image_1, image3)
_H2 = fun.ransac(source_points_2, destination_points_2)
mergered_image_2 = fun.combined_images(mergered_image_1, image3, _H2, focus=1, blend=True)
mergered_image_2 = fun.remove_black_background((mergered_image_2))
homography_matrix_list.append(_H2)
print(_H2)
show_image(mergered_image_2, axis=True)

In [ ]:
source_points_3, destination_points_3, matching_result_3 = fun.source_destination_points(mergered_image_2, image4)
_H3 = fun.ransac(source_points_3, destination_points_3)
mergered_image_3 = fun.combined_images(mergered_image_2, image4, _H3, focus=1, blend=False)
mergered_image_3 = fun.remove_black_background((mergered_image_3))
homography_matrix_list.append(_H3)
print(_H3)
show_image(mergered_image_3, axis=True)

In [ ]:
source_points_4, destination_points_4, matching_result_4 = fun.source_destination_points(mergered_image_3, image5)
_H4 = fun.ransac(source_points_4, destination_points_4)
mergered_image_4 = fun.combined_images(mergered_image_3, image5, _H4, focus=1, blend=True)
mergered_image_4 = fun.remove_black_background((mergered_image_4))
homography_matrix_list.append(_H4)
print(_H4)
show_image(mergered_image_4, axis=True)

## Final I4

In [ ]:
print((homography_matrix_list))
show_image(mergered_image_4, axis=False)

### path => I5

In [ ]:
path1 = '/content/drive/MyDrive/Colab Notebooks/content/Images/I5/DSC03002.JPG'
path2 = '/content/drive/MyDrive/Colab Notebooks/content/Images/I5/DSC03003.JPG'
path3 = '/content/drive/MyDrive/Colab Notebooks/content/Images/I5/DSC03004.JPG'
path4 = '/content/drive/MyDrive/Colab Notebooks/content/Images/I5/DSC03005.JPG'
path5 = '/content/drive/MyDrive/Colab Notebooks/content/Images/I5/DSC03006.JPG'
image1 = open_resize(path1, 600 , 400)
image2 = open_resize(path2, 600 , 400)
image3 = open_resize(path3, 600 , 400)
image4 = open_resize(path4, 600 , 400)
image5 = open_resize(path5, 600 , 400)
show_image(image1, axis=True)
show_image(image2, axis=True)
show_image(image3, axis=True)
show_image(image4, axis=True)
show_image(image5, axis=True)
homography_matrix_list = []

In [ ]:
source_points_1, destination_points_1, matching_result_1 = fun.source_destination_points(image1, image2)
_H1 = fun.ransac(source_points_1, destination_points_1)
mergered_image_1 = fun.combined_images(image1, image2, _H1, focus=2, blend=True)
mergered_image_1 = fun.remove_black_background((mergered_image_1))
homography_matrix_list.append(_H1)
print(_H1)
show_image(mergered_image_1, axis=True)

In [ ]:
source_points_2, destination_points_2, matching_result_2 = fun.source_destination_points(mergered_image_1, image3)
_H2 = fun.ransac(source_points_2, destination_points_2)
mergered_image_2 = fun.combined_images(mergered_image_1, image3, _H2, focus=2, blend=True)
mergered_image_2 = fun.remove_black_background((mergered_image_2))
homography_matrix_list.append(_H2)
print(_H2)
show_image(mergered_image_2, axis=True)

In [ ]:
source_points_3, destination_points_3, matching_result_3 = fun.source_destination_points(mergered_image_2, image4)
_H3 = fun.ransac(source_points_3, destination_points_3)
mergered_image_3 = fun.combined_images(mergered_image_2, image4, _H3, focus=2, blend=True)
mergered_image_3 = fun.remove_black_background((mergered_image_3))
homography_matrix_list.append(_H3)
print(_H3)
show_image(mergered_image_3, axis=True)

In [ ]:
source_points_4, destination_points_4, matching_result_4 = fun.source_destination_points(mergered_image_3, image5)
_H4 = fun.ransac(source_points_4, destination_points_4)
mergered_image_4 = fun.combined_images(mergered_image_3, image5, _H4, focus=1, blend=True)
mergered_image_4 = fun.remove_black_background((mergered_image_4))
homography_matrix_list.append(_H4)
print(_H4)
show_image(mergered_image_4, axis=True)

## Final I5

In [ ]:
print((homography_matrix_list))
show_image(mergered_image_4, axis=False)

### path => I6

In [ ]:
path1 = '/content/drive/MyDrive/Colab Notebooks/content/Images/I6/1_1.JPG'
path2 = '/content/drive/MyDrive/Colab Notebooks/content/Images/I6/1_2.JPG'
path3 = '/content/drive/MyDrive/Colab Notebooks/content/Images/I6/1_3.JPG'
path4 = '/content/drive/MyDrive/Colab Notebooks/content/Images/I6/1_4.JPG'
path5 = '/content/drive/MyDrive/Colab Notebooks/content/Images/I6/1_5.JPG'
image1 = open_resize(path1, 600 , 400)
image2 = open_resize(path2, 600 , 400)
image3 = open_resize(path3, 600 , 400)
image4 = open_resize(path4, 600 , 400)
image5 = open_resize(path5, 600 , 400)
show_image(image1, axis=True)
show_image(image2, axis=True)
show_image(image3, axis=True)
show_image(image4, axis=True)
show_image(image5, axis=True)
homography_matrix_list = []

In [ ]:
source_points_1, destination_points_1, matching_result_1 = fun.source_destination_points(image1, image2)
_H1 = fun.ransac(source_points_1, destination_points_1)
mergered_image_1 = fun.combined_images(image1, image2, _H1, focus=2, blend=True)
mergered_image_1 = fun.remove_black_background((mergered_image_1))
homography_matrix_list.append(_H1)
print(_H1)
show_image(mergered_image_1, axis=True)

In [ ]:
source_points_2, destination_points_2, matching_result_2 = fun.source_destination_points(mergered_image_1, image3)
_H2 = fun.ransac(source_points_2, destination_points_2)
mergered_image_2 = fun.combined_images(mergered_image_1, image3, _H2, focus=2, blend=True)
mergered_image_2 = fun.remove_black_background((mergered_image_2))
homography_matrix_list.append(_H2)
print(_H2)
show_image(mergered_image_2, axis=True)

In [ ]:
source_points_3, destination_points_3, matching_result_3 = fun.source_destination_points(mergered_image_2, image4)
_H3 = fun.ransac(source_points_3, destination_points_3)
mergered_image_3 = fun.combined_images(mergered_image_2, image4, _H3, focus=2, blend=True)
mergered_image_3 = fun.remove_black_background((mergered_image_3))
homography_matrix_list.append(_H3)
print(_H3)
show_image(mergered_image_3, axis=True)

In [ ]:
source_points_4, destination_points_4, matching_result_4 = fun.source_destination_points(mergered_image_3, image5)
_H4 = fun.ransac(source_points_4, destination_points_4)
mergered_image_4 = fun.combined_images(mergered_image_3, image5, _H4, focus=2, blend=True)
mergered_image_4 = fun.remove_black_background((mergered_image_4))
homography_matrix_list.append(_H4)
print(_H4)
show_image(mergered_image_4, axis=True)

## Final I6

In [ ]:
print((homography_matrix_list))
show_image(mergered_image_4, axis=False)